In [158]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [159]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv('Resources/lending_data.csv')

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [160]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"]

# Separate the X variable, the features
X = lending_df.drop(columns=['loan_status'])

In [161]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [162]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [163]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [164]:
X_train.shape

(58152, 7)

In [165]:
X_test.shape

(19384, 7)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [166]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(solver= 'lbfgs', random_state=1)

# Fit the model using training data
model_fit = model.fit(X_train_scaled, y_train)

print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Training Data Score: 0.9942908240473243
Testing Data Score: 0.9936545604622369
Coefficients: [[-0.46595101 -1.02980934 -1.07065518  8.81123915  0.01787448  0.19587583
  -1.07065518]]
Intercept: [-12.33611749]


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [167]:
# Make a prediction using the testing data

predictions = model_fit.predict(X_test_scaled)
results_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

results_df




,Prediction,Actual
60914,0,0
36843,0,0
1966,0,0
70137,0,0
27237,0,0
...,...,...
45639,0,0
11301,0,0
51614,0,0
4598,0,0


In [168]:
results_df.value_counts()

Prediction  Actual
0           0         18652
1           1           609
            0           113
0           1            10
Name: count, dtype: int64

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [169]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
confusion_matrix(predictions, y_test)

array([[18652,    10],
       [  113,   609]], dtype=int64)

In [170]:
# Print the classification report for the model
from sklearn.metrics import classification_report
target_names = ["Healthy Loan", "High Risk Loan"]
print(classification_report(predictions, y_test, target_names=target_names))

                precision    recall  f1-score   support

  Healthy Loan       0.99      1.00      1.00     18662
High Risk Loan       0.98      0.84      0.91       722

      accuracy                           0.99     19384
     macro avg       0.99      0.92      0.95     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** According to the logistic regression model that was trained, tested, and ran the accuracy and precision of prediction was high for both healthy loans and high-risk loans.  Healthy loans being nearly all identified (99%) and all predicted (100%) with a larger support number (18862).  High-Risk loans were nearly all identified (98%), however only 84% of high-risk loans were actually predicted, they also had a smaller support sample pool of 722, which may have contributed to the lower rate of predictions, due to the smaller sample size.  Based on the overall metric scores of the model (accuracy - 99%, macro avg - 99%, 92%, & 95%, weighted average - 99%, 99%, & 99%). I would use this as a one tool to determine the viability of a loan along with other factors the would be provided at the time of the start of the loan application process.

Healthy Loan      
•	Precision - 0.99
•	recall - 1.00
•	f1-score - 1.00
•	support -18662
High Risk Loan
•	precision - 0.98
•	recall - 0.84
•	f1-score - 0.91
•	support - 722       

                                precision    recall   f1-score       support
      accuracy                                                0.99              19384
     macro avg          0.99      0.92            0.95              19384
  weighted avg       0.99      0.99            0.99               19384

Prediction  Actual
0                        0         18652
1                        1           609
                           0           113
0                        1            10
Name: count, dtype: int64


---